<h1 style="text-align: center">TIỀN XỬ LÍ DỮ LIỆU</h1>

In [2]:
import json
import requests

url = "https://raw.githubusercontent.com/khangdz43/SummarizeText/master/vnexpress_dataset.json"

response = requests.get(url)
data = response.json()

print("Loaded", len(data), "articles.")


Loaded 577 articles.


In [ ]:
import re # biểu thức chính quy
import json
from datasets import Dataset
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, TrainingArguments ,AutoTokenizer

# # load file json
# with open("vnexpress_dataset.json", "r", encoding="utf-8") as f:
#     all_data = json.load(f)
#     print(f"Loaded {len(all_data)} articles.")



In [17]:
# xem 1 dòng dữ liệu text đầu để đánh giá chất lượng dữ liệu
print(all_data[0]['article'])

Theo quyết định phê duyệt dự án đầu tư ngày 26/11 của Bộ Xây dựng, tuyến cao tốc sẽ được mở rộng trên tổng chiều dài hơn 96 km, điểm đầu ở nút giao Chợ Đệm, TP HCM; điểm cuối ở phía Bắc cầu Mỹ Thuận 2, tỉnh Đồng Tháp (Tiền Giang cũ). Trong đó, đoạn TP HCM - Trung Lương mở rộng từ 4 lên 8 làn xe, tốc độ thiết kế 120 km/h; đoạn Trung Lương - Mỹ Thuận nâng từ 4 lên 6 làn, tốc độ 100 km/h. Các hạng mục kèm theo gồm: đường gom, cầu - nút giao, trạm thu phí, điểm dừng nghỉ và hệ thống giao thông thông minh. Dự án có tổng mức đầu tư hơn 36.100 tỷ đồng theo hình thức BOT, do liên danh gồm Tập đoàn Đèo Cả, Công ty Đầu tư Hạ tầng Kỹ thuật TP HCM (CII), Tasco, Hoàng Long thực hiện, không dùng ngân sách. Công trình khởi công ngày 19/12, hoàn thành năm 2028; thời gian thu phí hoàn vốn 17 năm 3 tháng. Cao tốc TP HCM - Trung Lương - Mỹ Thuận là trục huyết mạch về Đồng bằng sông Cửu Long, giúp rút thời gian đi Mỹ Thuận xuống còn 1 giờ 45 phút, thay vì hơn 3 giờ khi đi quốc lộ 1. Đoạn TP HCM - Trung Lư

In [18]:
# lower case từng bài báo trong all_data
article_list = [item['article'].lower() for item in all_data]

In [19]:
# loại bỏ kí tự không cần thiết cho từng bài báo
clean_article_list = [re.sub(r"[^0-9a-zA-ZÀ-ỹ\s]", " ", article) for article in article_list]
clean_article_list = [re.sub(r"\s+", " ", article).strip() for article in clean_article_list]

In [20]:
# lưu data đã làm sạch vào file json (có cả summary và article)
clean_data = [{"article": article, "summary": all_data[i]["summary"]} for i, article in enumerate(clean_article_list)]
with open("clean_vnexpress_dataset.json", "w", encoding="utf-8") as f:
    json.dump(clean_data, f, ensure_ascii=False, indent=4)
print("Đã lưu clean_vnexpress_dataset.json thành công!")

Đã lưu clean_vnexpress_dataset.json thành công!


In [21]:
# Load dữ liệu sạch
with open("clean_vnexpress_dataset.json", "r", encoding="utf-8") as f:
    data = json.load(f)

article_list = []
summary_list = []

for item in data:
    article_list.append(item["article"])
    summary_list.append(item["summary"])

In [22]:

# Tokenize
tokenizer = AutoTokenizer.from_pretrained("google/mt5-small")

c:\Users\ACER\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\convert_slow_tokenizer.py:566: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [23]:

# định dạng dữ liệu
df = pd.DataFrame({
    "article": article_list,
    "summary": summary_list
})
df.head()
dataset = Dataset.from_pandas(df)
# format cái max length thằng summary với article
def preprocess(batch):
    inputs = tokenizer(batch["article"], truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(batch["summary"], truncation=True, padding="max_length", max_length=128)
    inputs["labels"] = labels["input_ids"]
    return inputs

dataset = dataset.map(preprocess, batched=True)


Map:   0%|          | 0/574 [00:00<?, ? examples/s]

In [24]:

# train model
# hướng Abstractive Summarization tạo ra câu từ mới còn extractive thì lấy nguyên câu có sẵn


model = AutoModelForSeq2SeqLM.from_pretrained("google/mt5-small")

from transformers import Trainer
args = TrainingArguments(
    "mt5-summarization",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=50,
    save_steps=500,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset,
)

trainer.train()


TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
# các chỉ số đánh giá model

In [ ]:
# thử nghiệm

def summarize(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    output = model.generate(**inputs, max_length=128)
    return tokenizer.decode(output[0], skip_special_tokens=True)
print(summarize(df["article"][0]))